<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/Market_neutral_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pya3

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pya3 import Aliceblue
from datetime import datetime, timedelta
import holidays


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "NSE.csv" /content/drive/MyDrive/

In [6]:
import pandas as pd

# Load the CSV file into a DataFrame
df1 = pd.read_csv('NSE.csv')

# Display the first few rows of the DataFrame to ensure it was loaded correctly
df1.head()


,Exch,Exchange Segment,Group Name,Symbol,Token,Instrument Type,Instrument Name,Formatted Ins Name,Trading Symbol,Lot Size,Tick Size
0,NSE,nse_cm,EQ,011NSETEST,14747,0,011NSETEST,011NSETEST-EQ,011NSETEST-EQ,1.0,0.05
1,NSE,nse_cm,EQ,GUJALKALI,1267,0,GUJARAT ALKALIES & CHEM,GUJALKALI-EQ,GUJALKALI-EQ,1.0,0.05
2,NSE,nse_cm,EQ,GUFICBIO,11606,0,GUFIC BIOSCIENCES LTD.,GUFICBIO-EQ,GUFICBIO-EQ,1.0,0.05
3,NSE,nse_cm,EQ,GTPL,21195,0,GTPL HATHWAY LIMITED,GTPL-EQ,GTPL-EQ,1.0,0.05
4,NSE,nse_cm,EQ,GTLINFRA,13745,0,GTL INFRA.LTD,GTLINFRA-EQ,GTLINFRA-EQ,1.0,0.05


In [9]:
import pandas as pd
import numpy as np
from datetime import datetime


def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def get_historical_data(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2020, 1, 1)
    to_datetime = datetime(2024, 9, 9)
    interval = "D"

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['close'] = df['close'].astype(float)

    return df['close']

def calculate_market_neutral_strategy(alice, reliance_token, ongc_token):
    # Retrieve historical data for Reliance and ONGC
    reliance_close = get_historical_data(alice, reliance_token)
    ongc_close = get_historical_data(alice, ongc_token)

    # Drop duplicate indices
    reliance_close = reliance_close[~reliance_close.index.duplicated(keep='first')]
    ongc_close = ongc_close[~ongc_close.index.duplicated(keep='first')]

    # Align the data by index
    data = pd.concat([reliance_close, ongc_close], axis=1, keys=['Reliance', 'ONGC']).dropna()

    # Calculate the spread between Reliance and ONGC
    data['Spread'] = data['Reliance'] - data['ONGC']

    # Calculate moving average and standard deviation of the spread
    data['Spread_Mean'] = data['Spread'].rolling(window=20).mean()
    data['Spread_Std'] = data['Spread'].rolling(window=20).std()

    # Generate buy/sell signals
    data['Long_Signal'] = data['Spread'] < (data['Spread_Mean'] - 2 * data['Spread_Std'])
    data['Short_Signal'] = data['Spread'] > (data['Spread_Mean'] + 2 * data['Spread_Std'])

    # Filter data with signals
    signals = data[(data['Long_Signal']) | (data['Short_Signal'])].copy()

    return signals


def print_signals(signals):
    print("\nMarket Neutral Strategy Signals for Reliance and ONGC:")
    for index, row in signals.iterrows():
        if row['Long_Signal']:
            print(f"{index.date()}: Buy ONGC, Sell Reliance (Spread: {row['Spread']:.2f})")
        elif row['Short_Signal']:
            print(f"{index.date()}: Sell ONGC, Buy Reliance (Spread: {row['Spread']:.2f})")

def save_to_csv(signals, filename="market_neutral_signals.csv"):
    signals.to_csv(filename, index=True)
    print(f"\nData saved to {filename}")

def main():
    # Initialize Aliceblue object and get credentials
    alice = initialize_alice()

    # Tokens for Reliance and ONGC
    reliance_token = 2885
    ongc_token = 2475

    # Calculate market neutral strategy
    signals = calculate_market_neutral_strategy(alice, reliance_token, ongc_token)

    # Print signals
    print_signals(signals)

    # Save results to CSV
    save_to_csv(signals)

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: ZOI3ENFehIigT6euig3ETUqO1FBGrt1G2kW7JOqt5MdZKn0MIj9R4T1F37IRPcqYQAq8mJzAxOdqodfjLNP1fFZhNx7jEpinId4NbicC6iS3i4ycYLKez8JN3i5Uq4Pa

Market Neutral Strategy Signals for Reliance and ONGC:
2020-01-28: Buy ONGC, Sell Reliance (Spread: 1355.15)
2020-01-30: Buy ONGC, Sell Reliance (Spread: 1328.15)
2020-01-31: Buy ONGC, Sell Reliance (Spread: 1302.70)
2020-02-01: Buy ONGC, Sell Reliance (Spread: 1279.00)
2020-02-03: Buy ONGC, Sell Reliance (Spread: 1282.05)
2020-02-28: Buy ONGC, Sell Reliance (Spread: 1236.70)
2020-03-02: Buy ONGC, Sell Reliance (Spread: 1227.05)
2020-03-09: Buy ONGC, Sell Reliance (Spread: 1039.60)
2020-03-11: Buy ONGC, Sell Reliance (Spread: 1081.85)
2020-03-12: Buy ONGC, Sell Reliance (Spread: 1000.50)
2020-04-22: Sell ONGC, Buy Reliance (Spread: 1298.25)
2020-04-24: Sell ONGC, Buy Reliance (Spread: 1349.40)
2020-06-19: Sell ONGC, Buy Reliance (Spread: 1673.65)
2020-06-22: Sell ONGC, Buy Reliance (Spread: 1661.30)
2020-07-13:

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Function to get user credentials
def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

# Function to initialize Alice Blue object
def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

# Function to get historical data for a given token
def get_historical_data(alice, token, from_date, to_date, interval='D'):
    instrument = alice.get_instrument_by_token('NSE', token)
    historical_data = alice.get_historical(instrument, from_date, to_date, interval)
    df = pd.DataFrame(historical_data)
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)

    # Remove duplicate index entries
    df = df[~df.index.duplicated(keep='first')]
    return df

# Function to run backtesting on market-neutral strategy
def backtest_market_neutral_strategy(alice, reliance_token, ongc_token):
    # Fetch historical data for both stocks
    from_date = datetime(2010, 1, 1)
    to_date = datetime.now()  # To now

    reliance_data = get_historical_data(alice, reliance_token, from_date, to_date)
    ongc_data = get_historical_data(alice, ongc_token, from_date, to_date)

    # Combine the data for both stocks
    data = pd.DataFrame({
        'Reliance_Close': reliance_data['close'],
        'Reliance_High': reliance_data['high'],
        'Reliance_Low': reliance_data['low'],
        'ONGC_Close': ongc_data['close'],
        'ONGC_High': ongc_data['high'],
        'ONGC_Low': ongc_data['low']
    }).dropna()

    # Set target and stoploss percentages
    target_pct = 0.01
    stoploss_pct = 0.005

    results = []

    # Loop through the data to apply the strategy
    for i in range(len(data) - 1):
        # Current day close price (entry price)
        reliance_entry_price = data['Reliance_Close'].iloc[i]
        ongc_entry_price = data['ONGC_Close'].iloc[i]

        # Next day data
        next_day_data = data.iloc[i + 1]

        # Calculate target and stoploss levels
        reliance_target = reliance_entry_price * (1 + target_pct)
        reliance_stoploss = reliance_entry_price * (1 - stoploss_pct)
        ongc_target = ongc_entry_price * (1 + target_pct)
        ongc_stoploss = ongc_entry_price * (1 - stoploss_pct)

        # Determine if target or stoploss is hit for Reliance
        reliance_outcome = None
        if next_day_data['Reliance_High'] >= reliance_target:
            reliance_outcome = 'Target Hit'
        elif next_day_data['Reliance_Low'] <= reliance_stoploss:
            reliance_outcome = 'Stoploss Hit'

        # Determine if target or stoploss is hit for ONGC
        ongc_outcome = None
        if next_day_data['ONGC_High'] >= ongc_target:
            ongc_outcome = 'Target Hit'
        elif next_day_data['ONGC_Low'] <= ongc_stoploss:
            ongc_outcome = 'Stoploss Hit'

        # Store the results for analysis
        results.append({
            'Date': data.index[i + 1],
            'Reliance Entry Price': reliance_entry_price,
            'ONGC Entry Price': ongc_entry_price,
            'Reliance Outcome': reliance_outcome,
            'ONGC Outcome': ongc_outcome
        })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Calculate summary statistics
    total_trades = len(results_df)
    reliance_target_hits = results_df[results_df['Reliance Outcome'] == 'Target Hit'].shape[0]
    reliance_stoploss_hits = results_df[results_df['Reliance Outcome'] == 'Stoploss Hit'].shape[0]
    ongc_target_hits = results_df[results_df['ONGC Outcome'] == 'Target Hit'].shape[0]
    ongc_stoploss_hits = results_df[results_df['ONGC Outcome'] == 'Stoploss Hit'].shape[0]

    # Summary of results
    print(f"Total Trades: {total_trades}")
    print(f"Reliance Target Hits: {reliance_target_hits}")
    print(f"Reliance Stoploss Hits: {reliance_stoploss_hits}")
    print(f"ONGC Target Hits: {ongc_target_hits}")
    print(f"ONGC Stoploss Hits: {ongc_stoploss_hits}")
    print(f"Reliance Success Rate: {(reliance_target_hits / total_trades) * 100:.2f}%")
    print(f"ONGC Success Rate: {(ongc_target_hits / total_trades) * 100:.2f}%")

    return results_df

# Main function to run the script
def main():
    # Initialize Aliceblue object
    alice = initialize_alice()

    # Define tokens for Reliance and ONGC
    reliance_token = 2885  # Token for Reliance
    ongc_token = 2475  # Token for ONGC

    # Run the backtest
    backtest_results = backtest_market_neutral_strategy(alice, reliance_token, ongc_token)

    # Save the backtest results to CSV
    backtest_results.to_csv("market_neutral_backtest_results.csv", index=False)
    print("\nBacktest results saved to 'market_neutral_backtest_results.csv'")

# Run the script
if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: ZOI3ENFehIigT6euig3ETUqO1FBGrt1G2kW7JOqt5MdZKn0MIj9R4T1F37IRPcqYQAq8mJzAxOdqodfjLNP1fFZhNx7jEpinId4NbicC6iS3i4ycYLKez8JN3i5Uq4Pa
Total Trades: 3653
Reliance Target Hits: 1819
Reliance Stoploss Hits: 1606
ONGC Target Hits: 2115
ONGC Stoploss Hits: 1447
Reliance Success Rate: 49.79%
ONGC Success Rate: 57.90%

Backtest results saved to 'market_neutral_backtest_results.csv'


In [23]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Function to get user credentials
def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

# Function to initialize Alice Blue object
def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

# Function to get historical data for a given token
def get_historical_data(alice, token, from_date, to_date, interval='D'):
    instrument = alice.get_instrument_by_token('NSE', token)
    historical_data = alice.get_historical(instrument, from_date, to_date, interval)
    df = pd.DataFrame(historical_data)
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)

    # Remove duplicate index entries
    df = df[~df.index.duplicated(keep='first')]
    return df

# Function to run backtesting on market-neutral strategy
def backtest_market_neutral_strategy(alice, reliance_token, ongc_token):
    # Fetch historical data for both stocks
    from_date = datetime(2010, 1, 1)
    to_date = datetime.now()  # To now

    reliance_data = get_historical_data(alice, reliance_token, from_date, to_date)
    ongc_data = get_historical_data(alice, ongc_token, from_date, to_date)

    # Combine the data for both stocks
    data = pd.DataFrame({
        'Reliance_Close': reliance_data['close'],
        'Reliance_High': reliance_data['high'],
        'Reliance_Low': reliance_data['low'],
        'ONGC_Close': ongc_data['close'],
        'ONGC_High': ongc_data['high'],
        'ONGC_Low': ongc_data['low']
    }).dropna()

    # Set target and stoploss percentages
    target_pct = 0.01
    stoploss_pct = 0.005

    results = []

    # Loop through the data to apply the strategy
    for i in range(len(data) - 1):
        # Current day close price (entry price)
        reliance_entry_price = data['Reliance_Close'].iloc[i]
        ongc_entry_price = data['ONGC_Close'].iloc[i]

        # Next day data
        next_day_data = data.iloc[i + 1]

        # Calculate target and stoploss levels
        reliance_target = reliance_entry_price * (1 + target_pct)
        reliance_stoploss = reliance_entry_price * (1 - stoploss_pct)
        ongc_target = ongc_entry_price * (1 + target_pct)
        ongc_stoploss = ongc_entry_price * (1 - stoploss_pct)

        # Determine if target or stoploss is hit for Reliance
        reliance_outcome = None
        if next_day_data['Reliance_High'] >= reliance_target:
            reliance_outcome = 'Target Hit'
        elif next_day_data['Reliance_Low'] <= reliance_stoploss:
            reliance_outcome = 'Stoploss Hit'

        # Determine if target or stoploss is hit for ONGC
        ongc_outcome = None
        if next_day_data['ONGC_High'] >= ongc_target:
            ongc_outcome = 'Target Hit'
        elif next_day_data['ONGC_Low'] <= ongc_stoploss:
            ongc_outcome = 'Stoploss Hit'

        # Store the results for analysis
        results.append({
            'Date': data.index[i + 1],
            'Reliance Entry Price': reliance_entry_price,
            'ONGC Entry Price': ongc_entry_price,
            'Reliance Target': reliance_target,
            'Reliance Stoploss': reliance_stoploss,
            'ONGC Target': ongc_target,
            'ONGC Stoploss': ongc_stoploss,
            'Reliance Outcome': reliance_outcome,
            'ONGC Outcome': ongc_outcome
        })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Calculate summary statistics
    total_trades = len(results_df)
    reliance_target_hits = results_df[results_df['Reliance Outcome'] == 'Target Hit'].shape[0]
    reliance_stoploss_hits = results_df[results_df['Reliance Outcome'] == 'Stoploss Hit'].shape[0]
    ongc_target_hits = results_df[results_df['ONGC Outcome'] == 'Target Hit'].shape[0]
    ongc_stoploss_hits = results_df[results_df['ONGC Outcome'] == 'Stoploss Hit'].shape[0]

    # Summary of results
    print(f"Total Trades: {total_trades}")
    print(f"Reliance Target Hits: {reliance_target_hits}")
    print(f"Reliance Stoploss Hits: {reliance_stoploss_hits}")
    print(f"ONGC Target Hits: {ongc_target_hits}")
    print(f"ONGC Stoploss Hits: {ongc_stoploss_hits}")
    print(f"Reliance Success Rate: {(reliance_target_hits / total_trades) * 100:.2f}%")
    print(f"ONGC Success Rate: {(ongc_target_hits / total_trades) * 100:.2f}%")

    # Get the latest trade results
    if not results_df.empty:
        latest_trade = results_df.iloc[-1]

    return results_df

# Main function to run the script
def main():
    # Initialize Aliceblue object
    alice = initialize_alice()

    # Define tokens for Reliance and ONGC
    reliance_token = 236  # Token for Reliance
    ongc_token = 404  # Token for ONGC

    # Run the backtest
    backtest_results = backtest_market_neutral_strategy(alice, reliance_token, ongc_token)

    # Save the backtest results to CSV
    backtest_results.to_csv("market_neutral_backtest_results.csv", index=False)
    print("\nBacktest results saved to 'market_neutral_backtest_results.csv'")

# Run the script
if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: ZOI3ENFehIigT6euig3ETUqO1FBGrt1G2kW7JOqt5MdZKn0MIj9R4T1F37IRPcqYQAq8mJzAxOdqodfjLNP1fFZhNx7jEpinId4NbicC6iS3i4ycYLKez8JN3i5Uq4Pa
Total Trades: 3654
Reliance Target Hits: 1894
Reliance Stoploss Hits: 1572
ONGC Target Hits: 2347
ONGC Stoploss Hits: 1254
Reliance Success Rate: 51.83%
ONGC Success Rate: 64.23%

Backtest results saved to 'market_neutral_backtest_results.csv'
